# XGBoost

## Import and load data

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from scipy import stats
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV


In [ ]:
X_train = pd.read_csv('../data/processed/X_train_norm.csv')
y_train = pd.read_csv('../data/processed/y_train_norm.csv')
X_test = pd.read_csv('../data/processed/X_test_norm.csv')
test_id = X_test.Id
boosttest = X_test.copy().drop(columns=["Id"])

In [ ]:
boosttest

## Model: XGBoost

In [ ]:
# Initialize XGB model

xgbmodel = XGBRegressor(objective = "reg:squarederror",
                        booster = "gbtree",verbosity = 2, 
                        n_jobs = -1)

# Parameters for tuning
adjparams = {'learning_rate': np.linspace(0.05, 0.15, 10),
             'min_split_loss':[0, 1, 2], 'max_depth':[3, 4, 5, 6],
             "n_estimator":[1000, 1100, 1200], "min_child_weight":[1,2,3,4]}    

# god_params = {'max_depth':3,'learning_rate':0.1,'n_estimators':1000,
#               'reg_alpha':0.001,'reg_lambda':0.000001,
#               'min_child_weight':3}
# defaultparams = {'min_child_weight':1, "max_delta_step":0}
# xgbmodel.fit(X_train, y_train)
# xgbpredictions4 = np.expm1(xgbmodel4.predict(boosttest))
# print(xgbpredictions4)

# Parameter Tuning

In [ ]:
xgbgridsearch = GridSearchCV(estimator = xgbmodel, 
             param_grid = adjparams, 
             cv =10, 
             return_train_score=True, scoring = 'r2', verbose = 2)

In [ ]:
# train model
xgbgridsearch.fit(X_train, y_train)
xgbgridsearch.cv_results_
xgbgridsearch.best_params_

In [ ]:
mean_test_score = xgbgridsearch.cv_results_['mean_test_score']
std_test_score = xgbgridsearch.cv_results_['std_test_score']
mean_train_score = xgbgridsearch.cv_results_['mean_train_score']
std_train_score = xgbgridsearch.cv_results_['std_train_score']
print(max(mean_train_score))
print(max(mean_test_score))


# Prepare Submission CSV

In [ ]:
X_test

In [ ]:
xgbpredictions = np.expm1(xgbgridsearch.predict(boosttest))
print(xgbpredictions)

In [ ]:
xgbsubmission = pd.DataFrame()
xgbsubmission['Id'] = test_id
xgbsubmission['SalePrice']= pd.DataFrame(xgbpredictions)
xgbsubmission.to_csv("xgbsubmission",index = False)